In [85]:
# Render our plots inline
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import label_binarize

matplotlib.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)

train = pd.read_csv('/Users/Kseniya/diplom/src/text_preprocess/linis_csv/train.csv')
print train.shape
print "======="
test = pd.read_csv('/Users/Kseniya/diplom/src/text_preprocess/linis_csv/test.csv')
print test.shape

def target_three_classes(target_col):
    target_res = pd.Series()
    for row in target_orig:
        if row == -1 or row == 0 or row == 1:
            target_res.loc[len(target_res)] = row
            continue
        if row < -1:
            target_res.loc[len(target_res)] = -1
        else:
            target_res.loc[len(target_res)] = 1
    return target_res

# separate target column
# prepare for multiclass classification
targets = target_three_classes(train['target'])
target_orig = targets.copy()

targets = label_binarize(targets, classes=[-1, 0, 1])
train.drop('target', axis=1, inplace=True)

assert(len(target_orig.unique()) == 3)

# replace nan values with zero value
def data_preprocess(data):
    # Nan -> 0
    print data.shape
    new_data = data.fillna(0)
    print new_data.shape
    return new_data


train = data_preprocess(train)
print "========="
test = data_preprocess(test)
#train.isnull().sum().sum()

(2219, 52132)
(957, 52132)
(2219, 52131)
(2219, 52131)
(957, 52132)
(957, 52132)


In [89]:
# classification into 3 classes
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import cross_validation
from sklearn import svm
from sklearn.linear_model import BayesianRidge, LinearRegression,LogisticRegression

classifiers = [
    #{'type': "random_forest", 'classifier': RandomForestClassifier, 'params': {'n_estimators': 1000, 'n_jobs': 4}}, 
    {'type': "ada boost", 'classifier': AdaBoostClassifier, 'params': {'n_estimators': 15}},
    #{'type': "logistic regression", 'classifier': LogisticRegression, 'params': {
    #                                                                    'solver': 'newton-cg',
     #                                                                   'multi_class': 'ovr',
      #                                                                  'n_jobs': 4}},
    #{'type': "liner regression", 'classifier': LinearRegression, 'params': {
     #                                                               'copy_X': True,
      #                                                              'normalize': True,
       #                                                             'n_jobs': 4}},
    #{'type': "bayes ridge", 'classifier': BayesianRidge, 'params': {}},
    #{'type': "svm linear", 'classifier': svm.SVC, 'params': {'kernel': 'linear' }},
    #{'type': "svm poly 3", 'classifier': svm.SVC, 'params': {'kernel': 'poly', 'degree': 3 }},
    #{'type': "svm poly 2", 'classifier': svm.SVC, 'params': {'kernel': 'poly', 'degree': 2 }}, 
    #{'type': "svm linear", 'classifier': svm.SVC, 'params': {'kernel': 'sigmoid', 'coef0': 1 }}, 
]

In [87]:
def draw_roc_for_n_class(fpr, tpr, roc_auc, classes, func_name):        
    # Plot of a ROC curve for a specific class - "positive"
    plt.figure()
    for i in range(len(classes)):
        plt.plot(fpr[i], tpr[i], label=classes[i] + ': ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(func_name)
    plt.legend(loc="lower right")
    plt.show()

In [88]:
# prepare for multiclass classification
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc

def roc_auc_for_classifiers(classes, train_set, target_set):
    X_train, X_test, y_train, y_test = train_test_split(train_set, target_set, test_size=.3,
                                                    random_state=0)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    
    for c in classes:
        params = dict(c.get('params', {}))
        classifier = c['classifier']
        #classifier_cross_val = OneVsRestClassifier(classifier(**params))
        
        # cross validation
        #cross_val_res = cross_validation.cross_val_score(classifier_cross_val, train_set, target_set, n_jobs=4, scoring='roc_auc').mean()
        #print "{}: {}\n".format(c['type'], cross_val_res)
        
        # roc auc
        classifier = OneVsRestClassifier(classifier(**params))
        y_score = classifier.fit(X_train, y_train).predict(X_test)
        
        # Compute ROC curve and ROC area for each class
        for i in range(y_train.shape[1]):
            fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            
        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"]) 
        
        # draw plots
        draw_roc_for_n_class(fpr, tpr, roc_auc, ['negative', 'neitral', 'positive'], c['type'])        

In [91]:
import xgboost
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc
import numpy as np

def xgboost_test(train_set_orig, target_set_orig):
    params = {
        'max_depth': 6,
        'max_delta_step': 5,
        'base_score': 0.7,
        'n_estimators': 100,
        'min_child_weight': 1,
        'learning_rate': 0.02,
        'subsample': 0.7,
        'colsample_bytree': 0.6,
        'scale_pos_weight': 0.5
    }    

    xgb_clf = xgboost.XGBClassifier(missing=np.nan, nthread=4, seed=13, 
                                 n_estimators=int(params['n_estimators']),
                                 base_score=params['base_score'],
                                 max_depth=int(params['max_depth']),
                                 max_delta_step=int(params['max_delta_step']),
                                 min_child_weight=params['min_child_weight'],
                                 learning_rate=params['learning_rate'],
                                 subsample=params['subsample'],
                                 scale_pos_weight=params['scale_pos_weight'],
                                 colsample_bytree=params['colsample_bytree'])

    X_train, X_test, y_train, y_test = train_test_split(train_set_orig, target_set_orig,
                                                        test_size=.3, random_state=0)
    y_test = label_binarize(y_test, classes=[-1, 0, 1])

    print "fitting the model ... "
    y_score = xgb_clf.fit(X_train, y_train).predict_proba(X_test)
        
    print "compute roc-auc.."    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    # Compute ROC curve and ROC area for each class
    fpr[0], tpr[0], _ = roc_curve(y_test, y_score)
    roc_auc[0] = auc(fpr[0], tpr[0])
            
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"]) 
        
    # draw plots
    draw_roc_for_n_class(fpr, tpr, roc_auc, ['all'], "xgboost")        


In [92]:
y_score = xgboost_test(train, target_orig)

fitting the model ... 
done


In [5]:
# TMP: remove?

main_features = train.ix[:, : 18]
word_vec = train.ix[:, 18:]

print word_vec.shape
print main_features.shape

column_name = [i for i in range(word_vec.shape[1])]
word_vec.columns = column_name

(2219, 52113)
